In [9]:
import rasterio
from rasterio.enums import Resampling
from rasterstats import zonal_stats
import shapely
from pathlib import Path
import numpy as np
from IPython.display import HTML
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format

toulouse = shapely.Polygon(
    [[359326,4833160],
     [376735,4842547],
     [385238,4826271],
     [367914,4816946],
     [359326,4833160]]
)

arcachon = shapely.Polygon(
    [[629473,4959797],
     [649299,4960143],
     [649315,4940864],
     [630504,4940913],
     [629473,4959797]]
)

biarritz = shapely.Polygon(
    [[620357, 4825277],
     [628680, 4825402],
     [629181, 4802997],
     [610093, 4802622],
     [610155, 4808442],
     [620357, 4825277]]
)

brisbane = shapely.Polygon(
    [[494125, 6974741],
     [494125, 6974741],
     [518910, 6973355],
     [518910, 6973355],
     [518712, 6953715],
     [518712, 6953715],
     [493976, 6955348],
     [493976, 6955348],
     [494125, 6974741]]
)

buenosaires = shapely.Polygon(
    [
         [356354,6181219],
         [375609,6181386],
         [376276,6145542],
         [356937,6145459],
         [356354,6181219],
    ]
)

cantho = shapely.Polygon(
    [
         [574413,1117316],
         [593936,1117304],
         [593959,1103770],
         [574344,1103701],
         [574413,1117316],
    ]
)

helsinki = shapely.Polygon(
    [
         [378038,6677645],
         [397619,6676929],
         [397165,6664249],
         [377680,6664846],
         [378038,6677645],
    ]
)

lagos = shapely.Polygon(
    [
         [528578,733313],
         [547696,733545],
         [547542,705913],
         [528578,706068],
         [528578,733313],
    ]
)

cairo = shapely.Polygon(
    [
         [321171,3336141],
         [340490,3335740],
         [340162,3314598],
         [320588,3314926],
         [321171,3336141],
    ]
)

maros = shapely.Polygon(
    [
         [775303,9473671],
         [795021,9473362],
         [794849,9461765],
         [777230,9461971],
         [775303,9473671],
    ]
)

montpellier = shapely.Polygon(
    [
        [569263,4842348],
        [579275,4847307],
        [585825,4841974],
        [585170,4832523],
        [576562,4831868],
        [576936,4825880],
        [579181,4824102],
        [579369,4821576],
        [568982,4816804],
        [569263,4842348],
    ]
)

munich = shapely.Polygon(
    [
         [681767,5348957],
         [701333,5349752],
         [701955,5330290],
         [682666,5329512],
         [681767,5348957],
    ]
)

strasbourg = shapely.Polygon(
    [
        [393875,5382644],
        [398185,5383652],
        [411526,5383377],
        [411434,5373887],
        [409371,5372282],
        [408179,5365681],
        [403503,5367698],
        [404236,5369761],
        [402998,5371549],
        [396901,5369944],
        [395755,5374345],
        [397864,5375446],
        [393875,5382644],
    ]
)

nantes = shapely.Polygon(
    [
         [597911,5240048],
         [619604,5239798],
         [620103,5220682],
         [598410,5220848],
         [597911,5240048],
    ]
)
newyork = shapely.Polygon(
    [
         [582100,4516163],
         [590073,4516192],
         [590013,4503800],
         [582190,4503741],
         [582100,4516163],
    ]
)

paris = shapely.Polygon(
    [
         [443272,5423320],
         [462678,5423046],
         [462472,5404120],
         [443203,5404051],
         [443272,5423320],
    ]
)

portelisabeth = shapely.Polygon(
    [
         [358931,6255259],
         [376445,6255433],
         [372154,6242906],
         [378823,6236701],
         [359569,6236469],
         [358931,6255259],
    ]
)

rio = shapely.Polygon(
    [
         [670012,7479859],
         [676732,7479713],
         [689295,7460867],
         [669939,7460429],
         [670012,7479859],
    ]
)

sanfrancisco = shapely.Polygon(
    [
         [541991,4182570],
         [553709,4185286],
         [559529,4158202],
         [543155,4157426],
         [541991,4182570],
    ]
)


shanghai = shapely.Polygon(
    [
         [344856,3486292],
         [363657,3486033],
         [363139,3442206],
         [344337,3442206],
         [344856,3486292],
    ]
)

tianjin = shapely.Polygon(
    [
         [552371,4330867],
         [571582,4330867],
         [571674,4295401],
         [552648,4295401],
         [552371,4330867],
    ]
)

scale_factor = 1/10

cities = [
    ('toulouse', toulouse),
    ('arcachon', arcachon),
    ('biarritz', biarritz),
    ('brisbane', brisbane),
    ('buenos-aires', buenosaires),
    ('can-tho', cantho),
    ('helsinki', helsinki),
    ('lagos', lagos),
    ('le-caire', cairo),
    ('maros', maros),
    ('montpellier', montpellier),
    ('munich', munich),
    ('strasbourg', strasbourg),
    ('nantes', nantes),
    ('new-york', newyork),
    ('paris', paris),
    ('port-elisabeth',portelisabeth),
    ('rio-janeiro', rio),
    ('san-francisco', sanfrancisco),
    ('shanghai', shanghai),
    ('tianjin', tianjin)
]

digitanie = Path('/work/OT/ai4geo/DATA/DATASETS/DIGITANIE_v3')
#digitanie = Path('/data/DIGITANIE')

In [7]:
mean = np.array([0,0,0,0], dtype=np.float64)
count = np.array([0,0,0,0], dtype=np.float64)

for city, poly in cities:
    
    #city = city.capitalize()
    city = city.upper()
    
    print(f'processing {city}')

    with rasterio.open(digitanie/f"{city}/{city}.tif") as dataset:
        
        print(f'data shape {dataset.shape}')
        
        # resample data to target shape
        data = dataset.read(
            out_shape=(
                dataset.count,
                int(dataset.height * scale_factor),
                int(dataset.width * scale_factor)
            ),
            resampling=Resampling.bilinear
        )

        # scale image transform
        transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2])
        )
    
    for band in range(1, 5):
        
        stat = zonal_stats(
            poly, 
            np.clip(data[band-1], 0, 1),
            stats="count mean",
            affine=transform,
            band_num=band
        )[0]
        
        count[band-1] += stat['count']
        mean[band-1] += stat['count']*(stat['mean']- mean[band-1]) / count[band-1]
              
print(f'dataset mean: {mean}')

processing TOULOUSE


KeyboardInterrupt: 

In [10]:
from dl_toolbox.normalizations import stretch_to_minmax

stats_table = []

for city, poly in cities:
    
    #city = city.capitalize()
    city = city.upper()
    print(f'processing {city}')
    

    with rasterio.open(digitanie/f"{city}/{city}.tif") as dataset:
        
        print(f'data shape {dataset.shape}')
        
        # resample data to target shape
        data = dataset.read(
            out_shape=(
                dataset.count,
                int(dataset.height * scale_factor),
                int(dataset.width * scale_factor)
            ),
            resampling=Resampling.bilinear
        )

        # scale image transform
        transform = dataset.transform * dataset.transform.scale(
            (dataset.width / data.shape[-1]),
            (dataset.height / data.shape[-2])
        )
    
    stat_vals = [city]
    
    for band in range(1, 5):
        
        stat=zonal_stats(
            poly, 
            data[band-1],
            stats="min max percentile_0.5 percentile_99.5",
            affine=transform,
            band_num=band
        )[0]
        
        stat_vals += [
            stat['min'],
            stat['max'],
            stat['percentile_0.5'],
            stat['percentile_99.5']
        ]
        
        stat2=zonal_stats(
            poly,
            stretch_to_minmax(data[band-1], stat['percentile_0.5'], stat['percentile_99.5']),
            stats='mean',
            affine=transform,
            band_num=band
        )[0]
        
        stat_vals += [stat2['mean']]
        
    stats_table.append(stat_vals)
    
cols = ['city']
for i in range(1,5):
    cols += [f'min_{i}', f'max_{i}', f'p05_{i}', f'p995_{i}', f'mean_{i}']
    
df_stats = pd.DataFrame(stats_table, columns = cols, dtype=float)
HTML(df_stats.to_html(index=False))

processing TOULOUSE
data shape (54062, 55600)


/work/OT/ai4usr/fournip/latest/lib/python3.8/site-packages/rasterstats/io.py:328: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


processing ARCACHON
data shape (45609, 45980)
processing BIARRITZ
data shape (52359, 46818)
processing BRISBANE
data shape (44188, 51195)
processing BUENOS-AIRES
data shape (81685, 41879)
processing CAN-THO
data shape (27967, 39931)
processing HELSINKI
data shape (27715, 41223)
processing LAGOS
data shape (56814, 40066)
processing LE-CAIRE
data shape (44261, 40971)
processing MAROS
data shape (24698, 41289)
processing MONTPELLIER
data shape (69683, 40416)
processing MUNICH
data shape (41208, 41530)
processing STRASBOURG
data shape (43327, 47019)
processing NANTES
data shape (48078, 51770)
processing NEW-YORK
data shape (25619, 16625)
processing PARIS
data shape (42280, 42674)
processing PORT-ELISABETH
data shape (40316, 41100)
processing RIO-JANEIRO
data shape (40584, 40899)
processing SAN-FRANCISCO
data shape (61958, 40745)
processing SHANGHAI
data shape (91285, 41597)
processing TIANJIN
data shape (73141, 40642)


/tmp/pbs.46512392.admin01/ipykernel_9342/2828862523.py:54: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  df_stats = pd.DataFrame(stats_table, columns = cols, dtype=float)


city,mean_1,min_1,max_1,p05_1,p995_1,mean_2,min_2,max_2,p05_2,p995_2,mean_3,min_3,max_3,p05_3,p995_3,mean_4,min_4,max_4,p05_4,p995_4
TOULOUSE,0.1059,0.0277,1.0329,0.0363,0.3343,0.1105,0.0480,0.9887,0.0578,0.3027,0.1188,0.0704,0.9420,0.0789,0.2909,0.2078,0.0194,1.0318,0.0246,0.4090
ARCACHON,0.0524,0.0235,0.9582,0.0272,0.3115,0.0754,0.0446,0.8634,0.0522,0.2606,0.0919,0.0643,0.8096,0.0721,0.2225,0.0772,0.0112,0.9580,0.0137,0.3704
BIARRITZ,0.0650,0.0235,0.9998,0.0280,0.2796,0.0836,0.0408,0.9111,0.0484,0.2514,0.0932,0.0618,0.8795,0.0674,0.2433,0.2429,0.0123,1.0260,0.0152,0.4711
BRISBANE,0.0944,0.0248,1.0026,0.0328,0.4374,0.1079,0.0434,0.9161,0.0533,0.4182,0.1152,0.0627,0.8846,0.0711,0.3908,0.2054,0.0150,1.0175,0.0165,0.4623
BUENOS-AIRES,0.1590,0.0306,0.9957,0.0435,0.5603,0.1583,0.0525,0.9209,0.0689,0.5123,0.1615,0.0736,0.8769,0.0857,0.4793,0.2465,0.0239,0.9985,0.0411,0.5985
CAN-THO,0.1219,0.0582,1.0211,0.0724,0.3088,0.1390,0.0794,0.9924,0.0976,0.3155,0.1454,0.0974,0.9440,0.1113,0.3074,0.2176,0.0718,1.0176,0.0828,0.3913
HELSINKI,0.4049,0.0570,2.5642,0.0603,0.7262,0.4106,0.0860,2.4344,0.0934,0.6883,0.4464,0.1299,2.3139,0.1417,0.7002,0.4361,0.0321,2.6740,0.0350,0.8409
LAGOS,0.1520,0.1085,0.6728,0.1107,0.2240,0.1649,0.1260,0.5447,0.1398,0.2172,0.1774,0.1452,0.4519,0.1606,0.2153,0.1825,0.0853,0.8339,0.0876,0.3040
LE-CAIRE,0.1687,0.0436,0.7730,0.0595,0.3619,0.1531,0.0621,0.7249,0.0818,0.2868,0.1507,0.0799,0.6694,0.0986,0.2461,0.2173,0.0272,0.9227,0.0419,0.4341
MAROS,0.1279,0.0299,0.8734,0.0439,0.3335,0.1296,0.0494,0.8435,0.0664,0.3003,0.1288,0.0689,0.8059,0.0817,0.2717,0.2395,0.0210,1.1070,0.0271,0.4463


In [11]:
cls_distrib_table = []

def class_distrib_tile(msk_path, num_class):
        
    with rasterio.open(msk_path) as file:
        labels = file.read(out_dtype=np.uint8)
    
    counts, bins = np.histogram(labels.flatten(), range(num_class+1))
    
    return counts, bins

NUM_CLASS = 10
for city, poly in cities:
    
    #city = city.capitalize()
    city = city.upper()
    print(f'processing {city}')
    
    msk_paths = sorted(
        [path.relative_to(digitanie) for path in Path(digitanie/city/'COS9').glob('*_mask.tif')],
        key=lambda x: int(x.stem.split('_')[1])
    )
    
    for msk in msk_paths:
        
        tile_distrib = [str(msk)]
        counts, bins = class_distrib_tile(digitanie/msk, NUM_CLASS)
        tile_distrib += list(counts)
        cls_distrib_table.append(tile_distrib)
    
cols = ['mask']+[f'class {i}' for i in range(NUM_CLASS)]
df_cls = pd.DataFrame(cls_distrib_table, columns = cols)
HTML(df_cls.to_html(index=False))

processing TOULOUSE
processing ARCACHON
processing BIARRITZ
processing BRISBANE
processing BUENOS-AIRES
processing CAN-THO
processing HELSINKI
processing LAGOS
processing LE-CAIRE
processing MAROS
processing MONTPELLIER
processing MUNICH
processing STRASBOURG
processing NANTES
processing NEW-YORK
processing PARIS
processing PORT-ELISABETH
processing RIO-JANEIRO
processing SAN-FRANCISCO
processing SHANGHAI
processing TIANJIN


mask,class 0,class 1,class 2,class 3,class 4,class 5,class 6,class 7,class 8,class 9
TOULOUSE/COS9/TOULOUSE_0_mask.tif,596125,438503,1085267,1751,727852,617714,93942,542789,84155,6206
TOULOUSE/COS9/TOULOUSE_1_mask.tif,672395,115418,685463,651586,428692,1052016,224175,318613,5441,40505
TOULOUSE/COS9/TOULOUSE_2_mask.tif,554994,616691,859937,2578,592175,552474,453009,526188,32204,4054
TOULOUSE/COS9/TOULOUSE_3_mask.tif,939296,118294,617876,0,889595,810644,309139,503060,0,6400
TOULOUSE/COS9/TOULOUSE_4_mask.tif,982006,19522,397873,475,1180447,655471,213659,599511,136194,9146
TOULOUSE/COS9/TOULOUSE_5_mask.tif,795205,42129,734723,0,824417,615682,444805,728637,7581,1125
TOULOUSE/COS9/TOULOUSE_6_mask.tif,470166,58601,795298,664147,148479,1525334,26496,450415,53740,1628
TOULOUSE/COS9/TOULOUSE_7_mask.tif,625077,217570,890693,23064,667644,922466,215798,610607,20497,888
TOULOUSE/COS9/TOULOUSE_8_mask.tif,1002554,19903,911582,0,655760,953573,97731,527277,0,25924
TOULOUSE/COS9/TOULOUSE_9_mask.tif,1006325,6862,101439,40958,1432877,781315,134642,689444,0,442


In [12]:
split_table = []

for city, poly in cities:
    
    #city = city.capitalize()
    city = city.upper()
    print(f'processing {city}')
    
    img_paths = sorted(
        [path.relative_to(digitanie) for path in Path(digitanie/city).glob('*_[0-9].tif')], 
        key=lambda x: int(x.stem.split('_')[-1])
    )
    
    for i, img in enumerate(img_paths):
        
        if i < 7: split=0
        elif i < 8: split=1
        else: split=2
        
        with rasterio.open(digitanie/img) as f:
            height, width = f.shape
            
        row = [img,0,0,width,height,split]
        split_table.append(row)
    
cols = ['img', 'col_off', 'row_off', 'width', 'height', 'split']
df_split = pd.DataFrame(split_table, columns = cols)
HTML(df_split.to_html(index=False))

processing TOULOUSE
processing ARCACHON
processing BIARRITZ
processing BRISBANE
processing BUENOS-AIRES
processing CAN-THO
processing HELSINKI
processing LAGOS
processing LE-CAIRE
processing MAROS
processing MONTPELLIER
processing MUNICH
processing STRASBOURG
processing NANTES
processing NEW-YORK
processing PARIS
processing PORT-ELISABETH
processing RIO-JANEIRO
processing SAN-FRANCISCO
processing SHANGHAI
processing TIANJIN


img,col_off,row_off,width,height,split
TOULOUSE/TOULOUSE_20130924_T_TOA_reproj-EPSG32631_cog_tile_0.tif,0,0,2048,2048,0
TOULOUSE/TOULOUSE_20130924_T_TOA_reproj-EPSG32631_cog_tile_1.tif,0,0,2048,2048,0
TOULOUSE/TOULOUSE_20130924_T_TOA_reproj-EPSG32631_cog_tile_2.tif,0,0,2048,2048,0
TOULOUSE/TOULOUSE_20130924_T_TOA_reproj-EPSG32631_cog_tile_3.tif,0,0,2048,2048,0
TOULOUSE/TOULOUSE_20130924_T_TOA_reproj-EPSG32631_cog_tile_4.tif,0,0,2048,2048,0
TOULOUSE/TOULOUSE_20130924_T_TOA_reproj-EPSG32631_cog_tile_5.tif,0,0,2048,2048,0
TOULOUSE/TOULOUSE_20130924_T_TOA_reproj-EPSG32631_cog_tile_6.tif,0,0,2048,2048,0
TOULOUSE/TOULOUSE_20130924_T_TOA_reproj-EPSG32631_cog_tile_7.tif,0,0,2048,2048,1
TOULOUSE/TOULOUSE_20130924_T_TOA_reproj-EPSG32631_cog_tile_8.tif,0,0,2048,2048,2
TOULOUSE/TOULOUSE_20130924_T_TOA_reproj-EPSG32631_cog_tile_9.tif,0,0,2048,2048,2


In [13]:
df_stats_duplic = df_stats.loc[df_stats.index.repeat(10)].reset_index(drop=True)
#df = pd.concat([df_split, df_cls, df_stats_duplic], axis=1)
#df

In [14]:
df_split.to_csv('../datamodules/splits/digitanie_all/split.csv', index=True)
df_stats_duplic.to_csv('../datamodules/splits/digitanie_all/stats.csv', index=True)
df_cls.to_csv('../datamodules/splits/digitanie_all/cls.csv', index=True)

In [3]:
import rasterio
from rasterio.enums import Resampling
from rasterstats import zonal_stats
import shapely
from pathlib import Path
import numpy as np
from IPython.display import HTML
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
from pathlib import Path
from dl_toolbox.datasources import Digitanie
from rasterio import windows

csvpath = Path('../datamodules/splits/digitanie_all')
splits = [[],[],[]]

df_split = pd.read_csv(csvpath/'split.csv', index_col=0)
df_stats = pd.read_csv(csvpath/'stats.csv', index_col=0)
df_cls = pd.read_csv(csvpath/'cls.csv', index_col=0)

In [10]:
total = list(df_cls.sum()[1:])
print(total)
coeffs_ce = [np.round(max(total)/c,1) for c in total]
print(coeffs_ce)
coeffs_bce = [np.round((sum(total) - c)/c,1) for c in total]
print(coeffs_bce)

[151718441, 59884859, 117976473, 66274510, 197897811, 192428964, 24724591, 104300675, 6812848, 727708]
[1.3, 3.3, 1.7, 3.0, 1.0, 1.0, 8.0, 1.9, 29.0, 271.9]
[5.1, 14.4, 6.8, 12.9, 3.7, 3.8, 36.3, 7.8, 134.4, 1267.0]


In [ ]:


for index, row in df_split.iterrows():
    
    minval = [df_stats.loc[index][f'min_{i}'] for i in range(1,5)]
    maxval = [df_stats.loc[index][f'max_{i}'] for i in range(1,5)]
    meanval = [df_stats.loc[index][f'mean_{i}'] for i in range(1,5)]
    
    splits[row['split']].append(
        Digitanie(
            image_path=digitanie/row['img'],
            zone=windows.Window(
                row['col_off'],
                row['row_off'],
                row['width'],
                row['height']
            ),
            label_path=digitanie/df_cls.loc[index]['mask'],
            minval=np.array(minval).reshape((-1, 1, 1)),
            maxval=np.array(maxval).reshape((-1, 1, 1)),
            meanval=np.array(meanval).reshape((-1, 1, 1)),
            nomenclature_name='main'
        )
    )

In [ ]:
splits[0][2]

Digitanie(bands=None, image_path=PosixPath('/data/DIGITANIE/Toulouse/Toulouse_EPSG32631_2.tif'), zone=Window(col_off=0, row_off=0, width=2048, height=2048), minval=array([[[0.0277401 ]],

       [[0.04800046]],

       [[0.07036241]],

       [[0.01943548]]]), maxval=array([[[1.03292692]],

       [[0.9886831 ]],

       [[0.94196028]],

       [[1.03176355]]]), meanval=array([[[0.10590577]],

       [[0.11046101]],

       [[0.11877787]],

       [[0.20776042]]]), label_path=PosixPath('/data/DIGITANIE/Toulouse/COS9/Toulouse_2.tif'), nomenclature_name='main', all_cls_counts=array([613036, 598855, 866077,   2598, 591878, 537950, 519767, 459166,
          948,   4029]))

In [47]:
with pd.ExcelWriter('/home/eh/fournip/digitanie_stats.xlsx') as writer:
    df.to_excel(writer, sheet_name='all')


In [16]:
import numpy as np
from dl_toolbox.utils import get_tiles
import shapely
import rasterio

DATA_POLYGON = shapely.Polygon(
    [[359326,4833160],
     [376735,4842547],
     [385238,4826271],
     [367914,4816946],
     [359326,4833160]]
)

def generate_polygon(bbox):
    """
    Generates a list of coordinates: [[x1,y1],[x2,y2],[x3,y3],[x4,y4],[x1,y1]]
    """
    return [[bbox[0],bbox[1]],
             [bbox[2],bbox[1]],
             [bbox[2],bbox[3]],
             [bbox[0],bbox[3]],
             [bbox[0],bbox[1]]]

with rasterio.open("/work/OT/ai4geo/DATA/DATASETS/DIGITANIE/Toulouse/Toulouse.tif") as f:
    h,w = f.shape
    full_in = False
    n = 0
    while not full_in:
        crop = rasterio.windows.Window(n, n, h-2*n, w-2*n)                            
        left, bottom, right, top = rasterio.windows.bounds(
            crop,
            transform=f.transform
        )
        crop_poly = shapely.Polygon(generate_polygon((left, bottom, right, top)))
        inter = shapely.intersection(crop_poly, DATA_POLYGON)
        area = shapely.area(inter) / shapely.area(crop_poly)
        full_in = (area>=.99)
        print(full_in)
        n += 1000
    n -= 1000
    mean = np.array([0,0,0,0], dtype=np.float64)
    i = 1
    for tile in get_tiles(w-2*n, h-2*n, 5000, col_offset=n, row_offset=n, cover_all=False):
        tile_img = f.read(window=tile)
        tile_mean = np.mean(tile_img, axis=(1,2))
        mean += (tile_mean - mean) / i
        i += 1
        print(mean)
    print(mean)
    #print(crop_img.shape)

False
False
False
False
False
False
False
False
False
False
False
False
False
True
[0.09849199 0.10335866 0.11288217 0.21531706]
[0.10470457 0.10885551 0.11775978 0.21399323]
[0.10903376 0.11242765 0.12168191 0.20664583]
[0.10844705 0.11138627 0.12069001 0.20740956]
[0.10557562 0.10966521 0.11909904 0.20698276]
[0.10710744 0.11109315 0.12032198 0.20756214]
[0.11020168 0.11371848 0.1227758  0.2059096 ]
[0.11283427 0.11598756 0.12498699 0.20361528]
[0.11156363 0.11483422 0.12402334 0.20355269]
[0.1106208  0.11421487 0.12348139 0.20388228]
[0.10903827 0.11329056 0.12251379 0.20506695]
[0.10861864 0.1129546  0.12211177 0.20540509]
[0.10802666 0.11251532 0.12180922 0.20359494]
[0.10809964 0.11251282 0.12184742 0.20328178]
[0.10789525 0.11236557 0.12185172 0.20181251]
[0.1078277  0.11238775 0.12180501 0.20202577]
[0.10830327 0.11284892 0.12225782 0.20118916]
[0.10843413 0.11267972 0.12215294 0.19965764]
[0.10836564 0.11236478 0.12190191 0.1976699 ]
[0.10763204 0.1118677  0.12150295 0.1969869